# IMPORTING LIBRARIES



In [271]:
import pandas as pd
import re
import nltk

In [304]:
# nltk.download('stopwords')
# from nltk.corpus import stopwords
# stopwords = stopwords.words('english')

  

# DATASETS

In [273]:
c1 = pd.read_csv("corona (1).csv")
c2 = pd.read_csv("corona.csv")
c3 = pd.read_csv("corona_virus.csv")
c4 = pd.read_csv("covid-19.csv")

h1 = pd.read_csv("USAID.csv")
h2 = pd.read_csv("WHO.csv")
h3 = pd.read_csv("World_Health.csv")

m1 = pd.read_csv("covid-19 cure.csv")
m2 = pd.read_csv("covid-19 vaccine.csv")
m3 = pd.read_csv("cure.csv")
m4 = pd.read_csv("fake news.csv")
m5 = pd.read_csv("vaccine.csv")

r1 = pd.read_csv("religion.csv")

In [274]:
#c1.shape,c2.shape,c3.shape,c4.shape, h1.shape,h2.shape,h3.shape, m1.shape,m2.shape,m3.shape,m4.shape,m5.shape, r1.shape

In [275]:
c2 = c2.drop(["extended_entities","withheld_in_countries"],axis=1)
h2 = h2.drop(["extended_entities","withheld_in_countries"],axis=1)
m2 = m2.drop(["withheld_in_countries"],axis=1)
m3 = m3.drop(["extended_entities","withheld_in_countries"],axis=1)

In [276]:
data = pd.concat([c1,c2,c3,c4,h1,h2,h3,m1,m2,m3,m4,m5,r1],axis=0)

In [277]:
data.shape

(35000, 30)

# SELECTING ROWS

In [278]:
data.isna().sum()

contributors                 35000
coordinates                  34987
created_at                       0
display_text_range               0
entities                         0
favorite_count                   0
favorited                        0
full_text                        0
geo                          34987
id                               0
id_str                           0
in_reply_to_screen_name      27565
in_reply_to_status_id        27855
in_reply_to_status_id_str    27855
in_reply_to_user_id          27565
in_reply_to_user_id_str      27565
is_quote_status                  0
lang                             0
metadata                         0
place                        34639
possibly_sensitive           30080
quoted_status                34056
quoted_status_id             31579
quoted_status_id_str         31579
retweet_count                    0
retweeted                        0
retweeted_status             14120
source                           1
truncated           

In [279]:
df = data[["created_at","full_text","favorite_count","id","is_quote_status","retweet_count","source","user"]]

# CLEANING

In [280]:
df = df.dropna(axis=0)

In [281]:
#df.isna().sum()

In [282]:
df.columns

Index(['created_at', 'full_text', 'favorite_count', 'id', 'is_quote_status',
       'retweet_count', 'source', 'user'],
      dtype='object')

In [283]:
df.head()

,created_at,full_text,favorite_count,id,is_quote_status,retweet_count,source,user
0,2020-06-22 10:14:11,RT @IYCOdisha: Students asked to give exams in...,0,1275009177480388612,False,90,"<a href=""http://twitter.com/download/android"" ...","{'id': 1206265793136091137, 'id_str': '1206265..."
1,2020-06-22 10:14:10,RT @NSUIAssam: Every day Corona cases are incr...,0,1275009174770864130,False,21,"<a href=""https://mobile.twitter.com"" rel=""nofo...","{'id': 1037590693357613056, 'id_str': '1037590..."
2,2020-06-22 10:14:09,RT @AdityaRajKaul: China gave the world Corona...,0,1275009172887592960,True,1496,"<a href=""http://twitter.com/download/iphone"" r...","{'id': 279855461, 'id_str': '279855461', 'name..."
3,2020-06-22 10:14:09,@Nigella_Lawson This is not going to help my c...,0,1275009170798862336,False,0,"<a href=""http://twitter.com/download/iphone"" r...","{'id': 19272739, 'id_str': '19272739', 'name':..."
4,2020-06-22 10:14:09,Even MLA's and officials are affected by Coron...,0,1275009170287104000,False,0,"<a href=""http://twitter.com/download/android"" ...","{'id': 1550694234, 'id_str': '1550694234', 'na..."


In [284]:
#extracting hashtags, handles, retweets
hashtags = [re.findall("#\w+_?\w+?",r) for r in df.full_text]
handles = [re.findall("@\w+_?\w+?",r) for r in df.full_text]
retweets = [re.findall("RT @\w+_?\w+?:",r) for r in df.full_text]

In [285]:
#converting 'created_at' to datetime
df.created_at = pd.to_datetime(df.created_at)

In [286]:
#removing URLs
clean = [re.sub("https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)"," ", r) for r in df.full_text]

#remove RT
clean = [re.sub("(RT @\w+)"," ",r) for r in clean]

#removing symbols
clean = [re.sub("[^\w]+"," ", r) for r in clean]

In [287]:
#cleaning text
df.full_text = clean

In [288]:
med = []
for r in df.source:
  med.append(r.split("\"")[4])

In [289]:
#removing other symbols
med = [re.sub("[></](Twitter)","",r) for r in med]
med = [re.sub("( for )","",r) for r in med]
med = [re.sub("(</a>)","",r) for r in med]
med = [re.sub("\s","",r) for r in med]

In [290]:
df['Medium'] = med

In [291]:
# #source column
# device = re.findall("Twitter for (\w+)",df.source.split())
# web = [re.findall("Twitter (Web App)",r) for r in df.source]

# for i in range(len(device)):
#   if(device[i]==[]):
#     device[i] = web[i]
# df['Medium'] = device

In [292]:
#Name and Location
loc = []
name = []
for r in df.user:
  loc.append(r.split("'")[17])
  name.append(r.split("'")[9])

In [293]:
#Names of user
df['Name'] = name

In [294]:
df_new = df.drop(["favorite_count","is_quote_status","source","user"],axis=1)

# FINAL DATASET

In [302]:
df_new.head()

,created_at,full_text,id,retweet_count,Medium,Name
0,2020-06-22 10:14:11,Students asked to give exams in the tough tim...,1275009177480388612,90,Android,Md Abu Zaid
1,2020-06-22 10:14:10,Every day Corona cases are increasing but sti...,1275009174770864130,21,WebApp,🎶Ujjval
2,2020-06-22 10:14:09,China gave the world Corona India gave the wo...,1275009172887592960,1496,iPhone,Shrikant Sahu
3,2020-06-22 10:14:09,Nigella_Lawson This is not going to help my c...,1275009170798862336,0,iPhone,Brooke Bannister
4,2020-06-22 10:14:09,Even MLA s and officials are affected by Coron...,1275009170287104000,0,Android,Karan.


In [297]:
#top 4 Medium

In [299]:
df_new.columns

Index(['created_at', 'full_text', 'id', 'retweet_count', 'Medium', 'Name'], dtype='object')

In [327]:
loc1 = []
for r in loc:
  if(r!=''):
    loc1.append(r)

In [333]:
# loc1

In [329]:
location = pd.DataFrame(loc1,columns=["Location"])

In [330]:
location.to_csv("location.csv")